# StackOverflow Data Analysis


Si è voluto formulare uno studio relativo al dataset di StackOverflow disponibile su Kaggle al seguente indirizzo: 
    https://www.kaggle.com/stackoverflow/stackoverflow
    
Stack Overflow è un sito di domande e risposte relative al mondo della programmazione. E' nato nel 2008 e si sostanzia in una piattaforma a partecipazione attiva in quanto ogni domanda e ogni risposta è valutata dalla community stessa, il che porta a perdere o guadagnare reputazione in base al proprio contributo, risultando in diversi privilegi.
Al 2019, il conteggio degli utenti supera i 16 milioni.
    
Il dataset comprende i dati della piattaforma dal 2008 ad oggi, risultante in più di 600 milioni di osservazioni distribuite su 16 tabelle, ognuna delle quali costituita da minimo 15 colonne. 
Il dataset fornisce diversi tipi di dati: dati relativi agli utenti, alle domande e risposte date dagli stessi, i post pubblicati e la totalità dei tag utilizzati sino ad oggi.

## Google BigQuery
Essendo un datasert particolarmente ampio e corposo l'accesso agli stessi dati è stato posibile grazie ad un servizio fornito da Google chiamato BigQuery.

BigQuery è un servizio RESTful erogato come PaaS (Platform as a Service) che offre accesso a grandi moli di dati, andando a sgravare la responsabilità di computazione dei dati in questione nei confronti del client.
Al fine di consultare i dati desiderati, BigQuery permette l'utilizzo di Standard SQL per formulare query che verranno eseguite sul cloud, utilizzando la potenza computazionale offerta dalle macchine Google, risultando in query anche piuttosto complesse venire eseguite con una latenza relativamente minima.

Nel nostro caso particolare l'utilizzo del servizio si è limitato alla richiesta di specifiche query al fine di ottenere i dati desiderati, ma lo stesso offre diverse feature sintetizzabili come segue:
- **Gestione dei dati** => possibilità di creare, eliminare strutture quali tabelle e views. E' possibile importare i dati in formato CSV, Parquet, Avro e JSON dalla piattaforma Google Storage.
- **Query** => query espresse in Standard SQL rendendo possibile l'accesso a dati desiderati.
- **Access Control** => BigQuery non permette di effettuare richieste di alcun tipo se non ad entità autenticate.
- **Machine Learning** => infine il serizio rende disponibile la creazione ed esecuzione di modelli di Machine Learning tramite query SQL

## Data Gathering
Come accennato in precedenza, l'uso di BigQuery involve l'utilizzo di un linguaggio di query come SQL, permettendo così l'accesso a grandi moli di dati come nel nostro caso. Quindi prima dell'effettiva fase di analisi, ci si è concentrati sul processo di raccolta dei dati e della loro importazione in un formato utilizabili, Pandas DataFrame nel nostro caso.
Di seguito verranno esposte le funzionalità di maggior importanza.


### Autenticazione
Per utilizzare il servizio BigQuery è necessario anzitutto essere autenticati e disporre di un account sulla piattaforma di Google Cloud. Di seguito si necessita la creazione di una chiave di autenticazione, in formato JSON nel nostro caso, disponibile al download una volta creato l'account Google Cloud e da includere nel progetto per essere utilizzata come segue:

In [ ]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage #import external packages

client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)

Come si denota, viene anzitutto creata un' istanza di client dalla chiave in formato JSON fornita e instaurata una connessione bidirezionale, in grado di ricevere ed inviare dati al server remoto su cui si effettuano le richieste. 
Inoltre è necessario creare una referenza al dataset desiderato attraverso il nome e al progetto di cui fa parte e da essa ottenere l'effettiva istanza del dataset.

### Interrogazione e conversione a DataFrame

E' possibile dunque scrivere la query ed eventualmente fornire un limite di risultati desiderati, in quanto si ricorda come BigQuery offre solamente 1 TB mensile di dati scaricabili.

In [ ]:
sample_size = 1000000 #--> 1 MLN

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE DATETIME(creation_date) < DATETIME(2020, 3,30,0,0,0)
    ORDER BY r
    LIMIT ''' + str(sample_size)

Utilizzando la funzione result() si effettua la richiesta secondo la query specificata ed infine attraverso la funzione to_dataframe() è possibile convertire i risultati ottenuti in un DataFrame Pandas pronto per una successiva analisi

In [ ]:
all_question = (
    client.query(query).result()
    .to_dataframe(bqstorage_client=storage_client)
)

### Salvare e leggere un DataFrame

Trovandosi a lavorari con grandi quantità di dati sarebbe stato alquanto scomodo effettuare una query ogni volta senza mai persistere i dati ottenuti, anche e soprattutto per il vincolo di quantità imposto da BigQuery. 
Si è deciso dunque di salvare ogni DataFrame ottenuto dopo aver testato il corretto funzionamento di ogni query


In [ ]:
all_question.to_csv('tags_dataframe.csv') #si salva il DataFrame in formato CSV nella cartella di progetto corrente

read_df = pd.read_csv('tags_dataframe.csv') #si legge il file e si ottiene il DataFrame

## Analysis

L'intera analisi è basata su domande di fondo che ci siamo posti inizialmente.
Quest'ultime avevano l'obiettivo di far chiarezza su un particolare aspetto del dataset e allo stesso tempo portare un'analisi esplorativa di aspetti da noi ritenuti interessanti. 
Principalmente ci si è concentrati su analisi relative agli utenti e l'influenza che essi hanno sull'environment "sociale" della piattaforma, basandosi sulla reputazione e al contributo di ogni utente evidenziabile in domande e risposte date.

Infine si è voluto dare spazio a degli off topic relativi ad interessi personali come i framework per lo sviluppo web

### Analisi Utente

#### Quanti utenti contribuiscono alla comunità ? 

In [ ]:
#PASUUUUUUUUUUUUUUUUUUUUUAAAAAAAAAAAAAAAAAAAAAAA

#### E' meritocratica la società di StackOverflow ?

Seguendo le considerazioni fatte in precedenza relativamente al contributo medio di ogni singolo utente e come è bilanciato il sostentamento della piattoforma in base alla loro attività (numero di domande e risposte), si è voluto approfondire ulteriormente la questione in ambito utenti provando inizialmente a suddividere ogni utente in diverse categorie di reputazione. 
La reputazione su StackOverflow consiste in un numero che meglio dovrebbe indicare quanto la community si fida di un utente e delle sue risposte. Guadagnando reputazione si dispongono particolari privilegi, vedi
moderator tools https://meta.stackexchange.com/help/privileges/moderator-tools.
La reputazione è guadagnata rispondendo in maniera utile a determinate domande e, dall'altra parte, ponendo domande interessanti.

Azioni principali (+):
- **Domanda contrassegnata come utile** => +10
- **Risposta contrassegnata come utile** => +10
- **Risposta contrassegnata come accettata** => +15 (+2 all'utente che la accetta)
- **Modifica contrassegnata come accettata** => +2 (max: +1000)

Azioni principali (-):
- **Domanda contrassegnata come non utile** => -2
- **Risposta contrassegnata come non utile** => -2
- **Risposta contrassegnata come non utile da te** => -1
- **Uno dei post riceve 6 segnalazioni di spam o insulti** => -600

E' possibile guadagnare al massimo 200 punti reputazione al giorno.
Si lasciano qui di seguito ulteriori informazioni sull'assegnazione del punteggio reputazione https://stackoverflow.com/help/bounty

Di seguito a questi ragionamenti sorge spontaneo domandarsi come è a sua volta bilanciato il contributo di ogni macrocategoria di reputazione in quanto a domande e risposte date e se questo dato rispecchia la reputazione stessa, con utenti con un'alta reputazione e molte risposte date e, viceversa, utenti con bassa reputazione e parecchie domande accompagnate da poche risposte.

Al fine si necessitano dati relativo al numero di utenti, il numero di domande e risposte ognuno e il relativo punteggio di reputazione 

##### Gathering
Si è scritta una query che permettesse il raccoglimento dei sopracitati dati ed essendo i dati relativi ad utenti, domande e risposte contenuti in tabelle differenti, si è fatto uso di un JOIN.
Ogni utente però poteva anche non avere dato nè domande nè risposte durante la sua vita su StackOverflow, dunque si è dovuto utilizzare un JOIN che tenesse conto di questo particolare, un LEFT JOIN.
Infatti quest'ultimo ritorna tutte le entries dalla tabella di sinistra e solamente i relativi match nella tabella di destra. 

In [ ]:
question_answer_view = '''
                SELECT id AS user_id, reputation, asked, answered
                        FROM `bigquery-public-data.stackoverflow.users` users
                        LEFT JOIN(
                            SELECT owner_user_id AS user_id, COUNT(*) AS asked
                            FROM `bigquery-public-data.stackoverflow.posts_questions`
                            GROUP BY user_id
                        ) questions ON users.id = questions.user_id
                        LEFT JOIN(
                            SELECT owner_user_id AS user_id, COUNT(*) AS answered
                            FROM `bigquery-public-data.stackoverflow.posts_answers`
                            GROUP BY user_id
                        ) answers ON users.id = answers.user_id
                limit '''

Si ottiene dunque un dataset con tante entries quante gli utenti, ognuna con il relativo conteggio di domande e risposte. Il nostro obiettivo però è ottenere dei dati raggruppati per reputazione con il conteggio di domande e risposta a sua volta raggruppato per reputazione.
Volendo sfruttare quindi la potenza di BigQuery, o per semplice comodità, si è riutilizzata la query come subquery per una nuova comprensiva dell'aggregazione

In [1]:
aggregate_query = '''
            SELECT
                reputation AS reputation,
                COUNT(*) AS users,
                SUM(asked) AS questions,
                SUM(answered) AS answers
            FROM(
                SELECT id AS user_id, reputation, asked, answered
                            FROM `bigquery-public-data.stackoverflow.users` users
                            LEFT JOIN(
                                SELECT owner_user_id AS user_id, COUNT(*) AS asked
                                FROM `bigquery-public-data.stackoverflow.posts_questions`
                                GROUP BY user_id
                            ) questions ON users.id = questions.user_id
                            LEFT JOIN(
                                SELECT owner_user_id AS user_id, COUNT(*) AS answered
                                FROM `bigquery-public-data.stackoverflow.posts_answers`
                                GROUP BY user_id
                            ) answers ON users.id = answers.user_id
            )
 group by reputation
 '''

aggregate = (
    client.query(aggregate_query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

aggregate.fillna(0)

NameError: name 'client' is not defined

Il dataset ottenuto non rispecchia ancora i nostri bisogni; come si vede ora i dati sono raggruppati sì per reputazione ma per ogni singolo valori
edi essa. Si necessita un ulteriore raggruppamento basato su range di reputazione

Nota particolare è il fatto che avendo utilizzato un LEFT JOIN eventuali dati mancanti dalle tabelle di destra (in questo caso domande e risposte) vengono riportate come NaN. Per ovviare questo comportamento si sostituisce ogni NaN con 0, attraverso la funzione .fillna(0)

Per comodità da ora in poi si utilizzeranno funzionalità presenti in Pandas

##### Analysis

In [ ]:
def reputationToCategory(x):
    if(x >= 1 and x <= 100):
        return "Usurpers"
    elif(x > 100 and x <= 1000):
        return "Slaves"
    elif(x > 1000 and x <= 10000):
        return "Lords"
    elif(x > 10000 and x <= 100000):
        return "Grandmasters"
    else: return "Gods"
        
    

aggregate['reputation'] = aggregate['reputation'].map(reputationToCategory)

Ogni punteggio di categoria è convertito nella relativa classe (nomi simpatici per evidenziare ulteriormente il ruolo). Si sono scelte range di categoria ascendenti per rispecchiare meglio la distribuzione di reputazione

Infine non resta che utilizzare la funzione di raggruppamento di Pandas per ottenere il DataFrame desiderato, con i dati raggruppati per range di reputazione

In [2]:
reputation_df = aggregate.groupby(['reputation']).sum()

NameError: name 'aggregate' is not defined

I dati ottenuti rappresentano la totalità dei dati relativi presenti nel dataset, quindi un semplice conteggio. Al fine di visualizzare quanto ottenuto in maniera grafica si è voluto normalizzare i dati rispetto al totale e fornirli in forma percentuale.
Si fa uso delle seguenti funzioni

In [ ]:
def normalizeUsersCount(x):
    totalUsers = df_plot['users'].sum()
    return (x * 100) / totalUsers
def normalizeQuestionsCount(x):
    totalQuestions = df_plot['questions'].sum()
    return (x * 100) / totalQuestions
def normalizeAnswersCount(x):
    totalAnswers = df_plot['answers'].sum()
    return (x * 100) / totalAnswers

df_plot['users'] = df_plot['users'].map(normalizeUsersCount)
df_plot['questions'] = df_plot['questions'].map(normalizeQuestionsCount)
df_plot['answers'] = df_plot['answers'].map(normalizeAnswersCount)

Infine non resta che plottare i grafici e visualizzare i dati, confrontandoli con le aspettative iniziali

In [3]:
fig = go.Figure(data=[go.Pie(labels=df_plot["reputation"], values=df_plot["users"], pull=[0.3, 0.2, 0.15, 0.1, 0])])
fig.update_layout(title={
        "x":0.5,
        "y":0.95,
        "text":"Users of Stack Overflow based on reputation categories",
        "xanchor":"center",
        "yanchor":"top"
    })
fig.update_traces(marker=dict(colors=["#d5d5d5", "#223943", "#1438de", "#c4a15a", "#007668"]))

fig.show()

NameError: name 'go' is not defined

In [ ]:
bar_df = df_plot.iloc[::-1]

colors_answer = ["#223943"] * 5
colors_question = ["#c4a15a"] * 5

bar = go.Figure(data=[
    go.Bar(name='Questions', x=bar_df["reputation"], y=bar_df["questions"], marker_color=colors_question),
    go.Bar(name='Answers', x=bar_df["reputation"], y=bar_df["answers"], marker_color=colors_answer)
])


bar.update_layout(barmode='group', xaxis={'categoryorder':'trace'}, title='Percentage of questions and answers per reputation category',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Percent of Questions/Anwers',
        titlefont_size=16,
        tickfont_size=14,
    ), bargroupgap=0.05)

bar.update_yaxes(tickprefix="%", showgrid=True)
bar.show()

Dal primo grafico si denota che la stragrande maggioranza di utenti presenti su StackOverflow(>90%) presenta un basso grado di reputazione (1 -100) a fronte del rimanente 8% circa di utenti su cui sono ripartite le rimanenti.
Analogamente alla prima analisi, si nota sempre come la società di StackOverflow sia propriamente di stampo oligarchico, in cui la vera forza trascinatrice (l'arte della risposta) della piattaforma risiede nelle mani di un gruppo di pochi, che, però, a quanto pare, risulta molto attivo.

Infatti si constata come il 90% delle domande sia concentrato nelle prime tre categorie di reputazione, dato dal fatto che molto probabilmente utenti con maggiore reputazione solitamente sono utenti autorevoli e consapevoli, con un ridotto bisogno di porre domande. Analogamente il 70% delle risposte date appartiene alle 3 categorie con reputazione più alta

Effettivamente, anche dopo aver compreso le prime analisi, ci si aspettava un risultato di questo tipo dove il potere di risposta propende verso categorie con maggiore reputazione, in una società che per la sua quasi interezza è rappresentata da utenti non contribuenti che effettuano relativamente poche risposte a fronte delle innumerevoli domande poste. L'istogramma infatti mostra una netta inversione di marcia; si nota inoltre come il bilanciamento quasi perfetto tra percentuale di domande fatte e risposte date si ha nella terza categoria (1000 - 10000)

### Off Topic

#### Le vecchie tecnologie stanno per essere dimenticate ? 